In [ ]:
import pandas as pd
import numpy as np
import collections
filepath  = 'FILEPATH'
df = pd.read_csv(filepath)

BUILD A DICTIONARY OF FIGHTERS AND THEIR FIGHTS

In [912]:
fwin = df[df.win == 1] #fights that had a winner
flist = pd.concat([fwin['F1'], fwin['F2']]).unique() 
fightdict = dict()
 

BUILD A DICTIONARY OF FIGHTER'S OPPONENTS IN A FORMAT THAT WILL MAKE SET OPERATIONS EASY

In [913]:
for fighter in flist:
    tempstore = []
    fights1 = fwin[fwin.F1 == fighter]
    fights2 = fwin[fwin.F2 == fighter]
    fights12 = fights1.append(fights2)
    fightdict[fighter] = fights12 #the full fighter dictionary (all data)
    

USE FIGHT DICTIONARIES TO FIND 3 FIGHT SEQUENCES, TAKING INTO CONSIDERATION FIGHTERS WHO HAVE FOUGHT MULTIPLE TIMES. ALL POSSIBLE FIGHT COMBINATIONS WILL BE USED.

FOR EXAMPLE, IF F1 vs F2 HAPPENED TWICE THEN THE TWO FIGHT SEQUENCES RECORDED WOULD BE:

[1] F1 vs F2 (1),    F2 vs F3,    F1 vs F3 

[2] F1 vs F2 (2),    F2 vs F3,    F1 vs F3


In [914]:
"""
 This code works by looking at each fight (F1 vs F2), and then searching through all fighters in fightdict
 and checking whether both fighters (F1 & F2) are contained in the fighter 3's fightdict
 If so, record the sequence in storage list
"""

storage = list()

for index, row in fwin.iterrows():
    
    ftr1 = row['F1']
    ftr2 = row['F2']

    for key in fightdict:
       
        if ((ftr1 in set(fightdict[key].F1) or ftr1 in set(fightdict[key].F2)) and (ftr2 in set(fightdict[key].F1) or ftr2 in set(fightdict[key].F2)) and ftr1!=key and ftr2!=key):                
            cont = fightdict[key]
            contlistf11 = fightdict[key].loc[fightdict[key].F1 == ftr1] 
            contlistf12 = fightdict[key].loc[fightdict[key].F2 == ftr1]
            sd = contlistf11.append(contlistf12) # recording all rows in fightdict[key] which contains ftr1
            contlistf21 = fightdict[key].loc[fightdict[key].F1 == ftr2]
            contlistf22 = fightdict[key].loc[fightdict[key].F2 == ftr2]
            sds = contlistf21.append(contlistf22) # recording all rows in fightdict[key] which contains ftr2
            
            for fight1,row1 in sd.iterrows():
                
                for fight2,row2 in sds.iterrows():
                    new = [index, fight1, fight2]    
                    storage.append(new)
                

ENSURING FIGHT SEQUENCES ARE NOT REPEATED

In [915]:

storageset = set([frozenset(storage[i]) for i in range(len(storage))]) #frozenset allows set of sets to work

# converting to set removes repeated fight combinations
storagefin = list(storageset) #convert back to list so its indexable for next step
storagefin = [list(storagefin[i]) for i in range(len(storagefin))] #converting each element into a list 
        

DETERMINING WHETHER A FIGHT SEQUENCE DEMONSTRATES THE TRANSITIVE PROPERTY AND CALCULATING RATIO

In [916]:

nomath = 0
math = 0

for i in range(len(storagefin)):
    set1 = [fwin.F1.loc[storagefin[i][0]], fwin.F1.loc[storagefin[i][1]], fwin.F1.loc[storagefin[i][2]]]
    set2 = [fwin.F2.loc[storagefin[i][0]], fwin.F2.loc[storagefin[i][1]], fwin.F2.loc[storagefin[i][2]]]
    
    if collections.Counter(set1) == collections.Counter(set2):
        nomath = nomath+1
    else:
        math = math+1
        
print(1-nomath/math)

0.8124625524266027
